In [9]:
import os
import json
from pprint import pprint
import pandas as pd
import sklearn
import nltk

In [5]:
data_dir = 'Data/pdf_json/'
filenames = os.listdir(data_dir)

papers=[]
for filename in filenames[:10]:
    file = data_dir + filename
    paper = json.load(open(file, 'rb'))
    body_texts = [body_text['text'] for body_text in paper['body_text']]
    text_joined=''
    for text in body_texts:
        text_joined += text
    papers.append(text_joined)


In [38]:
clean_df = pd.DataFrame(papers)
clean_df.head()

,0
0,"VP3, and VP0 (which is further processed to VP..."
1,"In December 2019, a novel coronavirus, SARS-Co..."
2,The 2019-nCoV epidemic has spread across China...
3,"Metagenomic sequencing, which allows us to dir..."
4,"Infectious bronchitis (IB), which is caused by..."


In [11]:
stopwords=set(nltk.corpus.stopwords.words('english'))

In [31]:
vectorizer = sklearn.feature_extraction.text.TfidfVectorizer(stop_words=stopwords)
vectorizer.fit_transform(clean_df[0])

<10x4223 sparse matrix of type '<class 'numpy.float64'>'
	with 7407 stored elements in Compressed Sparse Row format>

In [37]:
# vectorizer.vocabulary_
word_frequency = sorted(vectorizer.vocabulary_.items(), key=lambda x: x[1], reverse=True)

In [39]:
word_frequency[:10]

[('μg', 4222),
 ('δpk', 4221),
 ('å2', 4220),
 ('ºc', 4219),
 ('µm', 4218),
 ('µl', 4217),
 ('zoster', 4216),
 ('zinc', 4215),
 ('zhengzhou', 4214),
 ('zhejiang', 4213)]

In [ ]:
#TODO: begin trying to answer questions specified in timeline.
#TODO: try connecting to CORD-19 Azure database.